# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921673859208                   -0.69    5.8         
  2   -7.926163569988       -2.35       -1.22    1.0    214ms
  3   -7.926837933196       -3.17       -2.37    1.6    236ms
  4   -7.926861516104       -4.63       -3.05    2.8    256ms
  5   -7.926861648747       -6.88       -3.44    1.9    225ms
  6   -7.926861672141       -7.63       -3.85    2.0    254ms
  7   -7.926861680282       -8.09       -4.29    1.2    203ms
  8   -7.926861681701       -8.85       -4.77    1.8    219ms
  9   -7.926861681861       -9.79       -5.35    2.0    227ms
 10   -7.926861681871      -10.99       -5.90    2.0    232ms
 11   -7.926861681873      -11.95       -6.68    2.0    233ms
 12   -7.926861681873      -13.33       -7.77    2.6    279ms
 13   -7.926861681873   +  -15.05       -8.03    3.6    276ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921699638922                   -0.69    5.8         
  2   -7.926165809443       -2.35       -1.22    1.0    221ms
  3   -7.926834129724       -3.18       -2.37    1.5    230ms
  4   -7.926861504148       -4.56       -2.98    2.9    264ms
  5   -7.926861621952       -6.93       -3.30    1.9    263ms
  6   -7.926861664578       -7.37       -3.70    1.5    211ms
  7   -7.926861680586       -7.80       -4.43    1.1    203ms
  8   -7.926861681817       -8.91       -4.97    2.2    242ms
  9   -7.926861681854      -10.44       -5.16    1.8    225ms
 10   -7.926861681872      -10.74       -6.06    1.4    211ms
 11   -7.926861681873      -12.08       -7.10    2.5    277ms
 12   -7.926861681873      -14.05       -7.02    3.0    270ms
 13   -7.926861681873      -14.45       -7.67    1.0    202ms
 14   -7.926861681873   +  -14.57       -8.04    1.9    220ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921668249567                   -0.69    5.8         
  2   -7.926172181672       -2.35       -1.22    1.0    208ms
  3   -7.926840904444       -3.17       -2.37    1.6    234ms
  4   -7.926864902567       -4.62       -3.02    2.6    302ms
  5   -7.926865042597       -6.85       -3.34    2.0    222ms
  6   -7.926865077434       -7.46       -3.73    1.6    205ms
  7   -7.926865091679       -7.85       -4.38    1.2    193ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
